In [2]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

In [3]:
from google.colab import drive
drive.mount('/content/drive')
#data_matrix = pd.read_csv('/content/drive/My Drive/CSC373_DMP_Mayr_Jack/DMP_Final/college_basketball.csv')
data_matrix = pd.read_csv('/content/drive/My Drive/CSC373_DMP_Mayr_Jack/DMP_Final/Country-data.csv')
data_matrix

Mounted at /content/drive


,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200
...,...,...,...,...,...,...,...,...,...,...
162,Vanuatu,29.2,46.6,5.25,52.7,2950,2.62,63.0,3.50,2970
163,Venezuela,17.1,28.5,4.91,17.6,16500,45.90,75.4,2.47,13500
164,Vietnam,23.3,72.0,6.84,80.2,4490,12.10,73.1,1.95,1310
165,Yemen,56.3,30.0,5.18,34.4,4480,23.60,67.5,4.67,1310


In [4]:
#data_matrix = data_matrix.drop(columns = ['POSTSEASON', 'TEAM', 'CONF', 'YEAR', 'SEED', 'G'])

normalized_data=(data_matrix.drop(columns = ['country'])-data_matrix.drop(columns = ['country']).min())/(data_matrix.drop(columns = ['country']).max()-data_matrix.drop(columns = ['country']).min())
#normalized_data['country'] = data_matrix['country']

In [5]:
normalized_data

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,0.426485,0.049482,0.358608,0.257765,0.008047,0.126144,0.475345,0.736593,0.003073
1,0.068160,0.139531,0.294593,0.279037,0.074933,0.080399,0.871795,0.078864,0.036833
2,0.120253,0.191559,0.146675,0.180149,0.098809,0.187691,0.875740,0.274448,0.040365
3,0.566699,0.311125,0.064636,0.246266,0.042535,0.245911,0.552268,0.790221,0.031488
4,0.037488,0.227079,0.262275,0.338255,0.148652,0.052213,0.881657,0.154574,0.114242
...,...,...,...,...,...,...,...,...,...
162,0.129503,0.232582,0.213797,0.302609,0.018820,0.063118,0.609467,0.370662,0.026143
163,0.070594,0.142032,0.192666,0.100809,0.127750,0.463081,0.854043,0.208202,0.126650
164,0.100779,0.359651,0.312617,0.460715,0.031200,0.150725,0.808679,0.126183,0.010299
165,0.261441,0.149536,0.209447,0.197397,0.031120,0.257000,0.698225,0.555205,0.010299


In [6]:
d = cdist(normalized_data, normalized_data)
d = d.flatten()
c1 = np.argsort(d)
c2 = np.sort(d)

data_len = len(normalized_data)
dist_list = pd.DataFrame()
dist_list['total_idx'] = c1
dist_list['idx_1'] = c1//data_len
dist_list['idx_2'] = c1%data_len
dist_list['dist'] = c2

filtered_dist_list = dist_list.drop_duplicates(subset=['dist'], keep='first')
filtered_dist_list = filtered_dist_list.drop(filtered_dist_list.loc[0])
filtered_dist_list.index = range(len(filtered_dist_list.index))

print(filtered_dist_list)
print(len(filtered_dist_list))
d = d[d>0]
print(d[0])
d
print(d)
print(len(d))
#d = sorted(d, reverse=False)

       total_idx  idx_1  idx_2      dist
0           6968     41    121  0.042979
1          12664     75    139  0.059012
2           4093     24     85  0.069718
3          15388     92     24  0.074491
4           4108     24    100  0.080048
...          ...    ...    ...       ...
15265      22277    133     66  1.977775
15266      15228     91     31  1.990490
15267      19004    113    133  2.023468
15268      15263     91     66  2.090027
15269      15310     91    113  2.097602

[15270 rows x 4 columns]
15270
0.8593844591671451
[0.85938446 0.74311097 0.44699259 ... 0.80784707 0.81790284 0.36781892]
27722


In [56]:
normalized_data['cluster'] = 0

def pandas_clustering_algo(data, sorted_dist):
  data.sort_index(inplace=True)
  num_clusters = len(data)
  curr_new_clust =  1
  print(len(sorted_dist))
  i=0
  while i < len(sorted_dist) and num_clusters > 2:
  #for i in range(len(sorted_dist)):
    print('iteration: ', i)
    print('number of unclustered points: ', len(data[data['cluster']==0]))
    print('min number of unclustered points: ', len(data)/10)
    if data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] == 0 and data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] == 0:
      print('first if')
      data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] = curr_new_clust
      data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] = curr_new_clust
      curr_new_clust += 1
      print('first if')
    elif data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] != 0 and data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] == 0:
      print('second if', data.loc[sorted_dist.loc[i,'idx_1'], 'cluster'])
      data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] = data.loc[sorted_dist.loc[i,'idx_1'], 'cluster']
      
    elif data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] != 0 and data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] == 0:
      print('third if', data.loc[sorted_dist.loc[i,'idx_1'], 'cluster'])
      data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] = data.loc[sorted_dist.loc[i,'idx_2'], 'cluster']
      
    elif data.loc[sorted_dist.loc[i,'idx_2'],'cluster'] != 0 and data.loc[sorted_dist.loc[i,'idx_1'],'cluster'] != 0:
      print('fourth if', sorted_dist.loc[i,'idx_1'])
      data.loc[data['cluster'] == sorted_dist.loc[i,'idx_1'] , 'cluster'] = sorted_dist.loc[i,'idx_2']
    print(data[data['cluster'] >0])
    i+=1
    a_set = set(data['cluster'])
    
    number_of_unique_values = len(a_set) + len(data[data['cluster'] == 0])
    print(number_of_unique_values)
    num_clusters = number_of_unique_values
  return data

In [ ]:
clustered_data = pandas_clustering_algo(normalized_data, filtered_dist_list)

Streaming output truncated to the last 5000 lines.
4
iteration:  12866
number of unclustered points:  0
min number of unclustered points:  16.7
fourth if 36
     child_mort   exports    health  ...      gdpp  cluster              country
0      0.426485  0.049482  0.358608  ...  0.003073      152          Afghanistan
1      0.068160  0.139531  0.294593  ...  0.036833      152              Albania
2      0.120253  0.191559  0.146675  ...  0.040365      152              Algeria
3      0.566699  0.311125  0.064636  ...  0.031488      152               Angola
4      0.037488  0.227079  0.262275  ...  0.114242       94  Antigua and Barbuda
..          ...       ...       ...  ...       ...      ...                  ...
162    0.129503  0.232582  0.213797  ...  0.026143      152              Vanuatu
163    0.070594  0.142032  0.192666  ...  0.126650      152            Venezuela
164    0.100779  0.359651  0.312617  ...  0.010299      152              Vietnam
165    0.261441  0.149536  0.2094

In [55]:
print(len(normalized_data[normalized_data['cluster'] ==0]))
a_set = set(normalized_data['cluster'])
number_of_unique_values = len(a_set)
print(number_of_unique_values)
print(a_set)
normalized_data['country'] = data_matrix['country']
print(normalized_data)

167
1
{0}
     child_mort   exports    health  ...      gdpp  cluster              country
0      0.426485  0.049482  0.358608  ...  0.003073        0          Afghanistan
1      0.068160  0.139531  0.294593  ...  0.036833        0              Albania
2      0.120253  0.191559  0.146675  ...  0.040365        0              Algeria
3      0.566699  0.311125  0.064636  ...  0.031488        0               Angola
4      0.037488  0.227079  0.262275  ...  0.114242        0  Antigua and Barbuda
..          ...       ...       ...  ...       ...      ...                  ...
162    0.129503  0.232582  0.213797  ...  0.026143        0              Vanuatu
163    0.070594  0.142032  0.192666  ...  0.126650        0            Venezuela
164    0.100779  0.359651  0.312617  ...  0.010299        0              Vietnam
165    0.261441  0.149536  0.209447  ...  0.010299        0                Yemen
166    0.391918  0.184556  0.253574  ...  0.011731        0               Zambia

[167 rows x 11 co